# To Do

* ~~parallelize experimental pipeline to make experiment iterations go faster on jakku~~
* experiment with different clustering approaches
    * different values for K in KMeans
    * with and without standardization of the training data for clustering
        * [a la this post](https://datascience.stackexchange.com/questions/6715/is-it-necessary-to-standardize-your-data-before-clustering)
    * DBScan or some other clustering approach
    * stratify data across activity classes to match labeled data
* experiment with uncertainty sampling instead of random sampling of personal data
* investigate clusters
    * How big are the clusters? Is it just a matter of size of the cluster?
    * are the clusters fairly homogenous/heterogenous with respect to
        * activity
        * user data (is a user's data mostly within one cluster, or many?)
* investigate why this method works great for some people, but not others
* What does this mean for designing human-in-the-loop systems?

# Methods

## Training Methods
***TODO***

### Impersonal and Personal Models
***TODO***

### Impersonal + Personal Models
***TODO***

### Cluster of Impersonal Data + Personal Model
***TODO***

## Validation Method
***TODO***



# Experiment 1

Comparing Universal + Personal data with varying amounts of personal data to 1 cluster of universal + personal data

### Results with v1.1 dataset : 
On average, the nearest cluster + personal data did about the same as the universal + personal data, but there were far fewer really bad scenarios especially with little labeled data. Also, for some individuals, this approach consistently proved to perform best.

### Results with v2.0 dataset :
nearest cluster + personal data did better than universal + personal data on average. Consistently served as improvement across many users.

### Results trained with v1.1 applied to v2.0:
Impersonal + Personal model performs nearly perfectly with every amount of personal data. Why is this?

** We assume that the labels are noisier in v2.0 data for two reasons: 1. The data was collected in the wild where users are more likely to input the correct activity incorrectly. 2. Training on this dataset and testing on a held out sample from the same dataset yields a result that is not promising. *How can this model be doing so well with noisy labels? You would think that there would be some more error due simply to user mislabeling* **


### Follow-up Question #1: 
*Q: What is different about the moments when this approach works and when it doesn't work?*

### Follow-up Question #2: 
*Q1 : Can we determine early in the use of the AR technology whether the personal, universal + personal, or cluster + personal approach will work best?*
* Reason about this question along with the Tong Yu, Yong Zhuang, Ole Mengshoel, Osman Yagan paper (Hybridizing Personal and Impersonal Machine Learning Models for Activity Recognition On Mobile Devices)

### Follow-up Question #3: 
For both the experiment in which the model was trained on v2.0 data and the experiment in which the model was trained on v1.1 and test on v2.0 data, the personal model did best for a surprisingly non-negligible amount of people when the dataset is smallest. This is counter-intuitive given that there is very very little data for the model to leverage here. 

*Q1 : Why does this work so well in the second two experiments?*

*Q2 : Why didn't this work well for the first experiment?*

### Follow-up Question #4:
Q : How do the class distributions in v2.0 differ from v1.1 and could that be affecting the differences in model performance? 

# Experiment 2
Using an uncertainty sampling approach to intelligently select data from available personal training set.